In [3]:
!pip install -r requirements_RAFT_finetuned_rag.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 1.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 151.6/151.6 kB 2.7 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of torchvision to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of torchvision to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 619.9/619.9 MB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.4/261.4 kB 25.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.6/44.6 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 939.3/939.3 kB 55.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 167.0/167.0 MB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━

In [4]:
import PyPDF2
import openai
import torch
import transformers
import streamlit as st # RAG
import pickle
import csv
import os

from time import time
from dotenv import load_dotenv # load your env
from langchain.llms import HuggingFacePipeline
from langchain.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains import RetrievalQA
from langchain.vectorstores import FAISS
from langchain.chat_models import ChatOpenAI
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings, HuggingFaceInstructEmbeddings
from langchain.chains import ConversationalRetrievalChain
from langchain.llms.base import LLM
from langchain.memory import ConversationBufferMemory
from peft import PeftConfig, PeftModel
from google.colab import userdata
from torch import cuda, bfloat16
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, T5ForConditionalGeneration


INFO:torch.distributed.nn.jit.instantiator:Created a temporary directory at /tmp/tmpq9zbsv5z
2024-07-25 12:48:54.002 INFO    torch.distributed.nn.jit.instantiator: Created a temporary directory at /tmp/tmpq9zbsv5z
INFO:torch.distributed.nn.jit.instantiator:Writing /tmp/tmpq9zbsv5z/_remote_module_non_scriptable.py
2024-07-25 12:48:54.014 INFO    torch.distributed.nn.jit.instantiator: Writing /tmp/tmpq9zbsv5z/_remote_module_non_scriptable.py


### Finetuned Custom class



In [5]:
class FineTuneModel:

  # Define the model text generation parameters
  generate_kwargs = dict(
          max_new_tokens= 512,
          num_beams=3,
          repetition_penalty= 2.2,
      )

  # Load the base model and tokenizer
  base_model = AutoModelForSeq2SeqLM.from_pretrained("google/flan-t5-base")
  tokenizer = AutoTokenizer.from_pretrained("google/flan-t5-base")

  # Load the fine-tuned PEFT model
  config = PeftConfig.from_pretrained("Orcawise/eu_ai_act_orcawise_july12")
  model = PeftModel.from_pretrained(base_model, "Orcawise/eu_ai_act_orcawise_july12")


  class PeftModelWrapper:

      def __init__(self, model, tokenizer, generate_kwargs):
          self.model = model
          self.tokenizer = tokenizer
          self.generate_kwargs = generate_kwargs
          self.task = "text2text-generation"

      def __call__(self, prompts):
          if not isinstance(prompts, list):
              prompts = [prompts]

          generated_texts =list()
          for prompt in prompts:
              if isinstance(prompt, dict):
                  prompt = prompt.get('input_text', '')

              # Tokenize input
              inputs_encoded = self.tokenizer(prompt, return_tensors="pt").input_ids

              # Generate output
              outputs = self.model.generate(
                                            input_ids=inputs_encoded,
                                            max_length=512,
                                            num_beams=3,
                                            repetition_penalty=2.2)

              # Decode output
              decoded_output = self.tokenizer.decode(outputs[0], skip_special_tokens=True)
              generated_texts.append(decoded_output)
          return [{"generated_text": text} for text in generated_texts]




config.json:   0%|          | 0.00/1.40k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

adapter_config.json:   0%|          | 0.00/603 [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/7.12M [00:00<?, ?B/s]

## Accessing the custom fine tuned class [build on top of flan-t5-base]

In [6]:
# Create the custom pipeline object of peft wrapper class
outside_class_fine_tuned_obj = FineTuneModel()
generate_kwargs = outside_class_fine_tuned_obj.generate_kwargs
tokenizer = outside_class_fine_tuned_obj.tokenizer
model = outside_class_fine_tuned_obj.model


custom_pipeline_obj = outside_class_fine_tuned_obj.PeftModelWrapper(model=model,
                                                                    tokenizer=tokenizer,
                                                                    generate_kwargs=generate_kwargs
                                     )
# Create HuggingFacePipeline using the custom pipeline
custom_llm_flan_t5_base = HuggingFacePipeline(pipeline=custom_pipeline_obj)


## Usage custom fine tuned class

In [7]:
input_text = "What is eu ai act."
response = custom_llm_flan_t5_base(input_text)
print(response)

The EU AI Act is a comprehensive framework for the development and deployment of AI systems in the European Union. Article 13 mandates that AI systems must be developed and deployed in accordance with the EU AI Act, including those designed to protect individuals' rights, freedoms, and fundamental rights. These systems must comply with the EU AI Act’s requirements for transparency, accountability, and fairness, as well as any other requirements set out in the EU AI Act.


## Accessing the open ai key from secrets

In [8]:
openai.api_key = userdata.get('OPENAI_API_KEY')
os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')

## RAG Class with Vector store

In [9]:
class RAG:

  """
  Function to read vectorStore
  This vector is vector of 900 question and answers
  """
  def read_vector_store(self, vector_store_path):
    with open(vector_store_path, 'rb') as f:
      vectorstore = pickle.load(f)
      return vectorstore


  def get_relevant_documents_from_vector_store(self, vectorstore, question, k=1):
    """
    k is the custom parameters which can range from 1 to .....
    k means how many vector store chunks in int value.
    """
    retriever_object = vectorstore.as_retriever()
    relevant_docs = retriever_object.get_relevant_documents(question)[:k]
    context = " ".join([doc.page_content for doc in relevant_docs])
    return context


## Testing RAG

In [10]:
input_text = "What is eu ai act."
rag_obj = RAG()
vector_store_path = "vector_store.pickle"
vectorstore = rag_obj.read_vector_store(
    vector_store_path=vector_store_path
    )
response_RAG = rag_obj.get_relevant_documents_from_vector_store(
                                                                vectorstore=vectorstore,
                                                                question=input_text,
    )
print("Response from RAG is \n", response_RAG)

INFO:faiss.loader:Loading faiss with AVX2 support.
2024-07-25 12:50:13.782 INFO    faiss.loader: Loading faiss with AVX2 support.
INFO:faiss.loader:Successfully loaded faiss with AVX2 support.
2024-07-25 12:50:13.877 INFO    faiss.loader: Successfully loaded faiss with AVX2 support.


Response from RAG is 
 Instruction: What is the EU AI ACT, and how does it impact businesses?  
Response: The EU AI ACT, or European Union Artificial Intelligence Act, is an  act introduced by the 
European Commission for the purposes of regulating the development, deployment, and uses of artificial 
intelligence (AI) systems within the European Union (EU). It aims to establish a harmonised framework for
the regulation of AI, ensuring the balance of both encouraging innovation and the protection of 
fundamental rights are considered. The Act impacts businesses that use or sell AI, by imposing 
regulations on the development and deployment of AI systems, particularly those considered high-risk. 
Businesses will be required to comply with the provisions of the Act  and ensure to conduct themselves 
with  transparency and accountability when interacting with AI systems.  It will be necessary that they 
abide by the risk assessment requirements set out by the legislation by adopting measur

## RAFT [RAG + Fine tuned model]

In [11]:
class RAFT:

  # constructor or special method
  def __init__(self, rag_object, custom_llm_flan_t5_base, vector_store_path):
    self.rag_object = rag_object
    self.custom_llm_flan_t5_base = custom_llm_flan_t5_base
    self.vector_store_path = vector_store_path
    self.vectorstore = self.rag_object.read_vector_store(
                                vector_store_path=self.vector_store_path
    )

  def get_response_raft(self, question):
    response_rag = self.rag_object.get_relevant_documents_from_vector_store(
                                                                vectorstore=self.vectorstore,
                                                                question=question,
    )
    # sending the response from the RAG to the fine tuned model
    prompt = f"Context: {response_rag}\n\nQuestion: {question}\n"
    response_raft = self.custom_llm_flan_t5_base(prompt)
    return response_raft

In [12]:
input_text = "What is eu ai act."
vector_store_path = "vector_store.pickle"
raft_object = RAFT(
    rag_object=rag_obj,
    custom_llm_flan_t5_base=custom_llm_flan_t5_base,
    vector_store_path=vector_store_path
    )
raft_response = raft_object.get_response_raft(question=input_text)
print("Response from RAFT is \n", raft_response)

Response from RAFT is 
 The EU AI Act, or European Union Artificial Intelligence Act, is an act introduced by the European Commission for the purposes of regulating the development, deployment, and uses of artificial intelligence (AI) systems within the European Union (EU). It aims to establish a harmonised framework for the regulation of AI, ensuring the balance of encouraging innovation and the protection of fundamental rights are considered. Businesses will be required to comply with the provisions of the Act and ensure to conduct themselves with transparency and accountability when interacting with AI systems, particularly those considered high-risk.


In [13]:
def write_to_csv(question, response_rag, response_fine_llm, response_raft, csv_file_path):
    """
    question: string and is something asked by the user in real time
    response_rag: string and is the response from the RAG vector store
    response_fine_llm: string and is the response from the fine tuned flab t5 base
    response_raft: string and is the response from the RAFT [RAG+FT]
    in future: string and will be response from the RAPM [RAG + GPT3.5]
    output: csv file contains the response from the RAG, fine tuned model and RAFT
    """
    # Define the header
    headers = ['Question', 'Output_RAG', 'Output_FineTune_Model', 'Output_RAFT']

    # Check if the file exists to determine if we need to write the header
    file_exists = os.path.isfile(csv_file_path)

    # Open the file in append mode
    with open(csv_file_path, mode='a', newline='') as file_obj:
        writer = csv.writer(file_obj)

        # Write the header if the file is new
        if not file_exists:
            writer.writerow(headers)

        # Write the response
        writer.writerow([question, response_rag, response_fine_llm, response_raft])

## Real time questions using chatgpt

In [15]:
eu_ai_act_questions = [
                      "What is the EU AI Act?",
                      "What are the main objectives of the EU AI Act?",
                      "How does the EU AI Act classify AI systems?",
                      "What are the high-risk AI applications identified by the EU AI Act?",
                      "What obligations are imposed on providers of high-risk AI systems under the EU AI Act?",

]

### Using the RAFT to generate the output for real time questions

In [16]:
for i in range(len(eu_ai_act_questions)):
  response_rag = rag_obj.get_relevant_documents_from_vector_store(
                                                      vectorstore=vectorstore,
                                                      question=eu_ai_act_questions[i]
  )
  response_fine_llm = custom_llm_flan_t5_base(eu_ai_act_questions[i])
  response_raft = raft_object.get_response_raft(question = eu_ai_act_questions[i])
  write_to_csv(
      question=eu_ai_act_questions[i],
      response_rag=response_rag,
      response_fine_llm=response_fine_llm,
      response_raft=response_raft,
      csv_file_path='Responses.csv')

## Code to merge the csv file making sure that unique rows are inserted

In [13]:
import csv
import pandas as pd
import os

In [ ]:
current_path = os.getcwd()
file1 = os.path.join(current_path, 'model_responses_from_chatgpt_nolan.csv')
file2 = os.path.join(current_path, 'Responses_source_chatgpt_zaid.csv')
file3 = os.path.join(current_path, 'response_vijay.csv')
df1 = pd.read_csv(file1, header=None)
df2 = pd.read_csv(file2, header=None)
df3 = pd.read_csv(file3, header=None)
# Concatenate the dataframes
merged_df = pd.concat([df1, df2, df3], ignore_index=True)
columns=['Question', 'RAG', 'RAFT', 'FinetunedModel']
# Save the merged dataframe to a new file
merged_df.columns = columns
merged_df.to_csv('merged_responses.csv', index=False)
